## Initialisation

In [ ]:
import numpy as np
import optimizationFuncs as optim
import matplotlib.pyplot as plt
from importlib import reload

import cpso
import chaosGen as cg
CPSO_Optimizer = cpso.CPSO_Optimizer

## Reload if necessary

In [ ]:
cpso = reload(cpso)
CPSO_Optimizer = cpso.CPSO_Optimizer

## Initialising CPSO object

In [ ]:
# Number of dimensions
dims = 1

# Left boundary --> 5 dimensions, and xi=-100 is the left boundary each dimension (i=0,1,2,3,4)
llim = np.repeat (-20, dims)

# Right boundary --> 5 dimensions, and xi=100 is the right boundary for each dimension (i=0,1,2,3,4)
rlim = np.repeat (20, dims)

# Shape of the swarm --> 25 particles in 5-D space
shape = (25, dims)

# Chaotic generators
initgen = cg.InverseLE(shape).chaosPoints
# randgen = cg.InverseLE(shape, le=1.5).chaosPoints
randgen = lambda x : np.random.random_sample(shape)

# Initialise optimizer object
func = optim.anuwu
mizer = cpso.CPSO_Optimizer(func, llim, rlim, initgen, randgen, cache=True)

## Optimize

In [ ]:
gbest, agrad = mizer.optimize()
print (" --> Iterations taken\nOptima = {}".format(gbest))

## Lyapunov Exponent

In [ ]:
ip = 0
ps = [0]
pert = np.copy(mizer.pcache[ip])

for p in ps :
    pert[p] += 1e-5*np.random.rand(dims)

seed = (pert, mizer.vcache[ip], mizer.mcache[ip], mizer.pbcache[ip], mizer.gbcache[ip], 
        mizer.r1cache[ip:], mizer.r2cache[ip:])
prep, vrep, mrep, pbrep, gbrep = mizer.replay(seed)

for p in ps :
    diff = np.sqrt(np.sum(np.square(mizer.pcache[ip:,p] - prep[:,p]), axis=1))
    print (np.mean(np.log(diff) - np.log(diff[0])))

## Forward Trajectory

In [ ]:
from time import sleep
import matplotlib.animation as anim

pp = 1
lin = np.linspace(llim[0], rlim[0], 1000)    
fig, ax = plt.subplots(1,1)
ax.plot(lin, func(lin.reshape(-1,1)))

for x, pb, gb in zip(mizer.pcache[:,pp], mizer.pbcache[:,pp], mizer.gbcache[:]) :
    print(x[0], pb[0], gb[0])
    plt.plot(lin, func(lin.reshape(-1,1)))
    plt.plot (x, func(x.reshape(-1,1)), 'o')
    plt.pause(0.00001)
    input()